In [ ]:
inicio=4
zw= 11
win= 1
def agrupamentoWPP(data,sw, normalize, limiar1):
    import numpy as np
    from biosppy.signals import tools
    import pickle

    LMnn= []
    LM=[]
    #normalizado
    if normalize==1:
        datanorm=[]
        for i in range(len(data)):
            H= tools.normalize(signal=data[i], ddof=1)
            datanorm.append(H[0])
        #'''self join mp'''
        for a in range(len(data)):
            info=[]
            localm1=[]
            for m in range(sw[0],sw[1],sw[2]):
                    from biosppy.signals import tools
                    ind , prof =tools.signal_self_join(signal=datanorm[a], size= m, index=None, limit=None)


                    #'''minimos da matrix profile'''
                    desv= prof.std()
                    med= prof.mean()
                    ln = len(prof)
                    M= []
                    if limiar1 == 0:
                        limiar1= (med - (2*desv))
                    for c in range(ln -1):
                        if (prof[c - 1]>prof[c]<prof[c + 1]):
                            if( prof[c]< limiar1):
                                if(c%len(data[0])+m<len(data[0])):
                                    M.append(c)


                    #'''lista de motifs normalizados'''
                    LMnn= []
                    for e in range(len(M)):
                        f= data[a][M[e]:M[e]+m]
                        LMnn.append(f)
                    for z in range(len(LMnn)):
                        R= tools.normalize(signal=LMnn[z], ddof=1)
                        LM.append(R[0])
    
    
    
    #nao normalizado
    else:        
        for a in range(len(data)):
            for m in range(sw[0],sw[1],sw[2]):
                from biosppy.signals import tools
                ind , prof =tools.signal_self_join(signal=data[a], size= m, index=None, limit=None)


                #'''minimos da matrix profile'''
                desv= prof.std()
                med= prof.mean()
                ln = len(prof)
                M= []
                for c in range(ln -1):
                    if (prof[c - 1]>prof[c]<prof[c + 1]):
                        if( prof[c]<0.1):
                            if(c%len(data[0])+m<len(data[0])):
                                M.append(c)


                #'''lista de motifs nao normalizados'''
                for e in range(len(M)):
                    f= data[a][[M[e]:M[e]+m]
                    LMnn.append(f)
            print('quentidade de subsequencias',len(LMnn))

#2 tabela de distancia entre motifs
DistMotifs= []
DMp=[]
if normalize!=1:
    LM= LMnn
for i in range(len(LM)):
    for j in range(i+1,len(LM)):
        if(len(LM[i])<=len(LM[j])/2):
            eucli = tools.distance_profile(query=LM[i], signal=LM[j], metric='euclidean')
            eucli=eucli[0]
        elif(len(LM[i])/2>=len(LM[j])):
            eucli = tools.distance_profile(query=LM[j], signal=LM[i], metric='euclidean')
            eucli=eucli[0]
        else:
            
            if(len(LM[i])>=len(LM[j])):
                o=[]
                o= np.concatenate((LM[i], LM[i]), axis=0)
                eucli = tools.distance_profile(query=LM[j], signal=o, metric='euclidean')
                eucli= eucli[0]
                a=[]
                for b in range(len(LM[i])-len(LM[j])+1):
                    a.append(eucli[b])
                eucli= a
            elif(len(LM[i])<len(LM[j])):
                o=[]
                o= np.concatenate((LM[j], LM[j]), axis=0)
                eucli = tools.distance_profile(query=LM[i], signal=o, metric='euclidean')
                eucli= eucli[0]
                a=[]
                for b in range(len(LM[j])-len(LM[i])+1):
                    a.append(eucli[b])
                eucli= a
             
        DMp.append(np.min(eucli))
    DistMotifs.append(DMp)
    DMp= []
import pickle
l=DistMotifs
with open("distmotifs.txt", 'wb') as fp:
    pickle.dump(l, fp)

#3 Agrupamento - variando o raio para o agrupamento, guardando raio e medida de qualidade
ra=[]
sel=0
medidaqualidade= []
informacao=[]
for rai in range(1, 11, 1):
    with open('distmotifs.txt', 'rb') as fp:
        D= pickle.load(fp)
    raio= rai/10
    # agrupando motifs
    g=[]
    for a in range(len(D)):
        gg=[]
        c= a - 1
        h=0
        for u in range(len(g)):
            if c not in (g[u]):
                h=h+1
        if(h==len(g)):
            if D[a] != []:
                gg.append(a-1)
                for p in range(int(zw/win)-2):
                    b= np.argmin(D[a])
                    c= b + a
                    h=0
                    for u in range(len(g)):
                        if c not in (g[u]):
                            h= h + 1
                    if(h==len(g)):
                        gg.append(c)
                        D[a][b]= 50
                for d in range(len(D[a])):
                    if(D[a][d]<=(np.min(D[a])+ raio)):
                        c= a+d
                        h=0
                        for u in range(len(g)):
                            if c not in (g[u]):
                                h= h + 1
                        if(h==len(g)):
                            gg.append(c)
                g.append(gg)

    
    #comparação de distancias intra grupo
    ç=0
    o=[]
    gg= []
    for f in range(len(g)):
        if(len(g[f])<2):
            gg.append(f)
    for f in range(len (gg) - 1, -1, -1):
        del(g[gg[f]])

    for u in range(len(g)):
        i=[]
        for v in range(len(g[u])):
            j=[]
            for l in range(len(g[u])):
                if(g[u][v]!=g[u][l]):
                    if(g[u][v]<g[u][l]):
                        j.append(DistMotifs[g[u][v] + 1][g[u][l]- g[u][v] - 1])
                    else:
                        j.append(DistMotifs[g[u][l] + 1][g[u][v]- g[u][l] - 1])
            a=0
            for n in range(len(j)):
                a= a + j[n]
            a= a/len(j)
            i.append(a)
        o.append(i)
    medg=[]
    centroides=[]
    for r in range(len(o)):
        centroides.append(np.argmin(o[r]))
        medg.append(np.min(o[r]))
    me=0
    for r in range(len(medg)):
        me= me + medg[r]
    valorintra= me/len(o)

    #comparação de distâncias extra grupos
    centros=[]
    i=[]
    for l in range(len(centroides)):
        centros.append(g[l][centroides[l]])
    a=0
    for v in range(len(centros)):
        j=[]
        for l in range(len(centros)):
            if(centros[v]!=centros[l]):
                if(centros[v]<centros[l]):
                    j.append(DistMotifs[centros[v] + 1][centros[l]- centros[v] - 1])
                else:
                    j.append(DistMotifs[centros[l] + 1][centros[v]- centros[l] - 1])
        if j==[]:print(centros)
        a= min(j)
        i.append(a)
    a=0
    for n in range(len(i)):
        a= a + i[n]
    valorextra= a/len(i)   
    #medida de qualidade
    print('grupos',len(g))
    print('ve',valorextra,'vi', valorintra)
    mq= (valorextra - valorintra)/max(valorintra, valorextra)
    ra.append(raio)
    medidaqualidade.append(mq)
    print('medidade de qualidade para ', raio, ' : ', mq)
    
    #seleção representantes do melhor agrupamento
    representantes=[] 
    if mq>sel:
        sel=mq
        for i in range(len(centros)):
            representantes.append(LMnn[centros[i]])
        import pickle
        l=representantes
        with open("representantes.txt", 'wb') as fp:
            pickle.dump(l, fp)
informacao.append(ra)
informacao.append(medidaqualidade)
with open("informaçao.txt", 'wb') as fp:
    pickle.dump(informacao, fp)

#tabela de atributo valor
tabela=[]
with open('representantes.txt', 'rb') as fp:
        Rep= pickle.load(fp)
for i in range(len(cdata)):
    a=cdata[i]
    linha=[]
    for j in range(len(Rep)):
        b= Rep[j]
        eucli = tools.distance_profile(query=b, signal=a, metric='euclidean')
        eucli= eucli[0]
        ln = len(eucli)
        contador= 0
        for c in range(ln -1):
            if (eucli[c - 1]>eucli[c]<eucli[c + 1]):
                if( eucli[c]< 0.1):
                    contador= contador + 1
        linha.append(contador)
    tabela.append(linha)
with open("tabela atributo-valor.txt", 'wb') as fp:
    pickle.dump(tabela, fp)

In [25]:
data= cdata

def agrupamentoWPP(data,sw, normalize, limiar1):
    import numpy as np
    from biosppy.signals import tools
    import pickle

    LMnn= []
    LM=[]
    #normalizado
    if normalize==1:
        datanorm=[]
        for i in range(len(data)):
            H= tools.normalize(signal=data[i], ddof=1)
            datanorm.append(H[0])
        #'''self join mp'''
        for a in range(1):
            info=[]
            localm1=[]
            for m in range(sw[0],sw[1],sw[2]):
                    from biosppy.signals import tools
                    ind , prof =tools.signal_self_join(signal=datanorm[a], size= m, index=None, limit=None)


                    #'''minimos da matrix profile'''
                    desv= prof.std()
                    med= prof.mean()
                    ln = len(prof)
                    M= []
                    if limiar1 == 0:
                        limiar1= (med - (2*desv))
                    for c in range(ln -1):
                        if (prof[c - 1]>prof[c]<prof[c + 1]):
                            if( prof[c]< limiar1):
                                if(c%len(data[0])+m<len(data[0])):
                                    M.append(c)


                    #'''lista de motifs normalizados'''
                    LMnn= []
                    for e in range(len(M)):
                        f= data[a][M[e]:M[e]+m]
                        LMnn.append(f)
                    for z in range(len(LMnn)):
                        R= tools.normalize(signal=LMnn[z], ddof=1)
                        LM.append(R[0])
                        
                        
                        
                        
v,w=agrupamentoWPP(data,[4,5,1],1,0.1)

16


In [26]:
print(w)

[[-4, -5, -4, -6], [-4, -6, -4, -3], [-4, -3, -5, -6], [-7, -9, -10, -9], [-9, -8, -9, -8], [-7, -9, -11, -12], [-13, -12, -13, -12], [-12, -14, -15, -14], [-11, -13, -11, -10], [-10, -8, -6, -8], [-6, -8, -10, -11], [-11, -12, -11, -10], [-10, -9, -11, -13], [-11, -13, -11, -9], [-10, -11, -10, -12], [-10, -12, -14, -15]]


In [5]:
import os
os.chdir('C:\\Users\\Usuario\\Desktop\\wallace\\UNILA\\DNA IC')
import numpy as np
from biosppy.signals import tools
data2= open('splice.data.txt', 'r')
f= data2.readlines()
for i in range(len(f)):
    f[i]= f[i].split(',')
for i in range(len(f)):
    del(f[i][1])
for i in range(len(f)):
    f[i][1]=f[i][1].split()[0]
c=[]
for i in range(len(f)):
    b=[]
    if f[i][0]=='EI':
        b.append(0)
    elif f[i][0]=='IE':
        b.append(1)
    elif f[i][0]=='N':
        b.append(2)
    a=[]
    l=0
    for j in range(len(f[i][1])):
        if f[i][1][j]=='A':
            l= l-1
            a.append(l)
        elif f[i][1][j]=='C':
            l= l-2
            a.append(l)
        elif f[i][1][j]=='G':
            l= l+1
            a.append(l)
        elif f[i][1][j]=='T':
            l= l+2
            a.append(l)
        else:
            a.append(l)
    b.append(a)
    c.append(b)
cdata=[]
ctarg=[]

for i in range(len(c)):
    cdata.append(c[i][1])
    ctarg.append((c[i][0]))


In [4]:
a=[[0,0],[0,0,0]]
a[0]= [1,2,3,4,5]
a

[[1, 2, 3, 4, 5], [0, 0, 0]]